In [1]:
import numpy as np
import pandas as pd
import requests, bs4
import os
import base64
import io
from datetime import datetime
from furl import furl

In [2]:
def get_api(url):
    request = requests.get(url=url).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(request, 'html.parser')
    rows = xmlobj.findAll('item')
    for i, row in enumerate(rows):
        climate.loc[len(climate)] = [
          row.tm.string.split(" ")[0],
          row.tm.string.split(" ")[1],
          row.ta.string,
          row.ws.string,
          row.wd.string,
          row.hm.string,
          row.pv.string,
          row.td.string,
          row.pa.string,
          row.ps.string,
          row.vs.string,
          row.ts.string ]
    return rows


In [3]:
for _,_, files in os.walk(os.getcwd()):
    for file in files:
        if "xlsx" in file:
            filename = file
            print(filename)
            df = pd.read_excel(filename,sheet_name='계량정보데이터')
            df=df.drop(['일자'],axis=1)
            df=df.drop(['Unnamed: 26'],axis=1)
            df = df.dropna()
            df=df.reset_index(drop=True,inplace=False)
            df.columns=['날짜','00:00','01:00','02:00','03:00','04:00','05:00','06:00','07:00','08:00','09:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00']
            raw_all=pd.melt(df,id_vars=['날짜'])
            raw_all.columns=['DateTime','Hour','kwh']


            if raw_all.index.dtype == 'int64':
                raw_all.set_index('DateTime', inplace=True)
            raw_all=raw_all.reset_index()
            dt = raw_all.DateTime.str.slice(0,10)
            raw_all.drop('DateTime',axis=1,inplace=True)
            raw_all['dt'] = dt


            raw_all = raw_all[['dt','Hour','kwh']]
            raw_all = raw_all.sort_values(by=['dt','Hour']).reset_index(drop=True)

            climate = pd.DataFrame(columns=("dt","Hour","ta","ws","wd","hm","pv","td","pa","ps","vs","ts"))


            for dt in raw_all['dt'].unique():
                url = furl('http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList')
                url.args['serviceKey'] = 'xM0E+Y9pYfVPh7W2rCzqvDgzBtmQZcawC3y/ONSyZTWUU6Jw2ttLGbUlq9Sccb6DCZ3g3Ej57Sv0LsQHo8Q1/w=='
                url.args['numOfRows'] = '30'
                url.args['pageNo'] = '1'
                url.args['dataCd'] = 'ASOS'
                url.args['dateCd'] = 'HR'
                url.args['stnIds'] = '108'
                url.args['startDt'] = dt.replace('-','')
                url.args['startHh'] = '00'
                url.args['endDt'] = dt.replace('-','')
                url.args['endHh'] = '23'
                rows = get_api(url.url)


            df = pd.merge(raw_all, climate, on=['dt','Hour'])
            df['month'] = df.dt.str.slice(5,7)
            df.drop(columns=["ps","vs"],inplace=True)
            csv_file = file[:-4]+"csv"
            df.to_csv(csv_file,index=False)

RRMSE_제안용_㈜ 젬텍_0923994065_2021.11.04_32638.xlsx


ValueError: File is not a recognized excel file

In [ ]:
 df.to_csv(file,index=False)

In [ ]:
df

In [ ]:
df.to.csv(1.csv,index=false)